In [2]:
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import pandas as pd
import numpy as np
import math
import git
import datetime
import warnings
warnings.filterwarnings("ignore")

# Leemos una sola vez la base de datos por ejecución del programa
link ='https://github.com/Sud-Austral/Datos_Panama/blob/master/datacovidpa/Localiza%20PN%20v1.xlsx?raw=true'
data = pd.read_excel(link,'TODO CORREGIMIENTO')

# Prueba con un Corregimiento:
# data = data[data['Corregimiento'] == 'Alanje (Cabecera)']

# ejecutaremos éstas funciones corregimiento a corregimiento:

def cleanBefore(data):  
    # eliminamos filas que no importan y asignamos la primera como cabecera
    data.drop([0], inplace=True)
    data.columns = data.iloc[0]
    data.drop([1], inplace=True)
    #eliminamos la primera fila que es un NA
    our_data_filter = data.iloc[1:]
    # extraemos un subconjunto que llegue solo hasta el dia de hoy:
    now = datetime.datetime.now()
    our_data_filter = our_data_filter[our_data_filter['Fecha'] < now]
    # filtramos desde el primer dia en el que haya recuperados:
    our_data_filter = our_data_filter[our_data_filter_un['NuevosRecuperadosacum'] > 0]
    # extraemos solo las columnas que nos interesan:
    our_data_filter = our_data_filter[['CD_Corr', 'Corregimiento', 'Fecha', 'Nuevos Confirmados', 'Nuevos Fallecidos', 'Nuevos Recuperados', 'Población']]
    return our_data_filter
    
def transAAC(data_transAAC):
    data_transAAC.rename(columns={'Nuevos Confirmados': 'NuevosConfirmados', 'Nuevos Fallecidos': 'NuevosFallecidos',
                         'Nuevos Recuperados': 'NuevosRecuperados', 'Población': 'Poblacion'}, inplace=True)
    data_transAAC['NuevosConfirmados'] = data_transAAC.to_numeric(data_transAAC['NuevosConfirmados'])
    data_transAAC['NuevosFallecidos'] = data_transAAC.to_numeric(data_transAAC['NuevosFallecidos'])
    data_transAAC['NuevosRecuperados'] = data_transAAC.to_numeric(data_transAAC['NuevosRecuperados'])
    data_transAAC['Poblacion'] = data_transAAC.to_numeric(data_transAAC['Poblacion'])
    data_transAAC['NuevosConfirmados'] = data_transAAC['NuevosConfirmados']+data_transAAC['NuevosFallecidos']
    data_transAAC['NuevosConfirmadosacum'] = data_transAAC['NuevosConfirmados'].cumsum()
    data_transAAC['NuevosRecuperadosacum'] = data_transAAC['NuevosRecuperados'].cumsum()
    data_transAAC["Susceptibles"] = data_transAAC['Poblacion']-data_transAAC['NuevosConfirmadosacum']-data_transAAC['NuevosRecuperadosacum']
    data_transAAC["pob_log_Susceptibles"] = data_transAAC['Poblacion']*np.log(data_transAAC["Susceptibles"])
    return data_transAAC

def calculate(data_cal):
    lista_R0 = []
    y = our_data_filter_un_final["pob_log_Susceptibles"]
    for i in range(len(y)):
        y = our_data_filter_un_final["pob_log_Susceptibles"]
        x = our_data_filter_un_final["NuevosRecuperadosacum"]
        yy = y.head(i+1)
        xx = x.head(i+1)
        # Creamos el objeto de Regresión Lineal
        regresion_lineal = linear_model.LinearRegression()
        regresion_lineal.fit(xx.values.reshape(-1, 1),yy.values.reshape(-1, 1))
        r0 = -regresion_lineal.coef_
        lista_R0.append(r0)
    our_data_filter_un_final["r0"] = lista_R0
    return our_data_filter_un_final

def cleanAfter(data_second_r0):
    x = data_second_r0['r0'].str.replace("]]", "")
    y = x.str.replace("[", "")
    z_dataframe = pd.DataFrame(z)
    del data_second_r0['r0']
    data_second_r0['r0'] = z_dataframe
    return data_second_r0

def saveParcial(dt):
    #Dejar los generados archivos en la misma carpeta
    dt.to_csv ('test.csv', mode='a', index = False, header=False)
    return 

def R0PanamaPorPartes():
    columnas = "CD_Corr,Corregimiento,Fecha,NuevosConfirmados,NuevosFallecidos,NuevosRecuperados,Poblacion,NuevosConfirmadosacum,NuevosRecuperadosacum,Susceptibles,pob_log_Susceptibles,r0"
    f = open ('test.csv','wb')
    f.write(columnas.encode())
    f.close()
    data = recuperarData()
    for corregimiento in data["Corregimiento"].unique():
        try:
            saveParcial(limpiezaPost(calculo(preCalculo(data[data["Corregimiento"] == corregimiento]))))
        except:
            print(corregimiento)
    return


1,CD_Corr,Corregimiento,Fecha,NuevosConfirmados,NuevosFallecidos,NuevosRecuperados,Poblacion
2,80817,Pacora,2020-06-05 00:00:00,NaN,NaN,NaN,35056
3,80819,Tocumen,2020-06-05 00:00:00,NaN,NaN,NaN,91447
4,130101,Arraiján (Cabecera),2020-06-05 00:00:00,NaN,NaN,NaN,53634
5,80821,24 de Diciembre,2020-06-05 00:00:00,NaN,NaN,NaN,79798
6,81002,Belisario Porras,2020-06-05 00:00:00,NaN,NaN,NaN,55951
...,...,...,...,...,...,...,...
45155,70314,Villa Lourdes,2020-08-10 00:00:00,-100.0,-4.0,-20.0,1088
45156,20107,Virgen del Carmen,2020-08-10 00:00:00,-2.0,0.0,-2.0,10367
45157,50206,Yape,2020-08-10 00:00:00,-375.0,-8.0,-344.0,216
45158,50317,Zapallal,2020-08-10 00:00:00,-3.0,0.0,-2.0,2072
